In [1]:
!pip install deap
import random

from deap import base
from deap import creator
from deap import tools

You should consider upgrading via the '/Users/Hamid/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
#    Based on OneMax example in DEAP
#    Modified by Kevin Wilson 18/10/2020

#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.


#    example which maximizes the sum of a list of integers
#    each of which can be 0 or 1



creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Attribute generator 
#                      define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,7] (i.e. an integer in the range 0 to 7 with equal
#                      probability)
toolbox.register("attr_intcol", random.randint, 0, 7)

# Structure initializers
#                         define 'individual' to be an individual
#                         consisting of 8 'attr_int' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_intcol, 8)

# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# the goal ('fitness') function to be minimised
def evalOneMax(individual):
    # print(individual)
    horiz_penalty_value = 0
    diag_penalty_value = 0
    
    # Check for row number duplication
    for col_value in individual:
        num_duplicates = individual.count(col_value)
        # print("col_value " + str(col_value) + " num_duplicates " + str(num_duplicates))
        if num_duplicates > 1:
            horiz_penalty_value = horiz_penalty_value + num_duplicates


    # print("horizontal " + str(horiz_penalty_value))
    # Check diagonals
    for current_col in range(8):  # Remember upper limit of range not included
        for other_col in range(8):
            if individual[other_col] != individual[current_col]:
                distance = abs(current_col - other_col)
                on_diag_above = (individual[other_col] == individual[current_col] + distance)
                on_diag_below = (individual[other_col] == individual[current_col] - distance)
                if on_diag_above or on_diag_below:
                    diag_penalty_value += 1
    # print("diag " + str(diag_penalty_value))

#    # print chess board sideways to check diag_penalty_value
#    for col_value in individual:
#        print(col_value, end='')
#        print(" ", end='')
#        for x in range(8):
#            if x == col_value:
#                print("X", end='')
#            else:
#                print("+", end='')
#        print()

    inverse_fitness_value = horiz_penalty_value + diag_penalty_value
    return inverse_fitness_value, # to be minimised

#----------
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", evalOneMax)

# register the crossover operator
toolbox.register("mate", tools.cxOnePoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selTournament, tournsize=3)

#----------

def main():
    random.seed(64)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=100)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2
    
    print("Start of evolution")
    
    print(pop)

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    
    print(fitnesses)
    print(len(fitnesses))
    
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while g < 1000:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        print("  Evaluated %i individuals" % len(invalid_ind), end='')
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        count_zeros = fits.count(0)
        print("----------------")
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        #print("  Valid solutions %s" % count_zeros)
    
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

    # Display all valid board configurations found
    for ind in pop:
        if ind.fitness.values[0] == 0:
            print(ind, ind.fitness.values[0])


if __name__ == "__main__":
    main()

Start of evolution
[[7, 1, 6, 0, 3, 4, 3, 2], [1, 6, 7, 3, 0, 1, 6, 0], [1, 2, 1, 0, 4, 3, 3, 1], [0, 7, 6, 3, 4, 2, 6, 5], [4, 5, 7, 2, 0, 0, 3, 7], [2, 3, 3, 0, 4, 3, 3, 0], [4, 1, 2, 4, 3, 3, 3, 2], [4, 6, 7, 4, 5, 2, 4, 2], [2, 4, 0, 2, 0, 1, 1, 0], [2, 5, 4, 6, 7, 0, 7, 5], [0, 0, 5, 2, 1, 5, 4, 2], [6, 6, 5, 2, 5, 6, 0, 0], [4, 2, 6, 2, 3, 7, 4, 7], [1, 5, 4, 4, 0, 6, 5, 1], [1, 7, 5, 7, 1, 0, 1, 0], [1, 6, 6, 1, 6, 2, 2, 4], [5, 1, 3, 0, 5, 3, 2, 6], [1, 3, 6, 2, 0, 4, 4, 6], [2, 7, 0, 5, 4, 6, 7, 5], [2, 3, 3, 5, 4, 2, 7, 0], [2, 0, 5, 5, 2, 3, 4, 6], [4, 4, 0, 6, 4, 7, 5, 0], [0, 5, 6, 4, 6, 0, 2, 7], [1, 7, 6, 0, 2, 6, 7, 7], [7, 7, 0, 5, 6, 4, 0, 2], [3, 3, 6, 0, 6, 3, 7, 1], [0, 5, 2, 3, 6, 4, 3, 4], [2, 6, 0, 4, 1, 0, 6, 6], [7, 4, 2, 7, 7, 6, 5, 6], [5, 1, 5, 7, 6, 2, 5, 4], [1, 1, 0, 1, 2, 0, 7, 7], [0, 0, 1, 5, 7, 2, 0, 2], [1, 3, 7, 3, 0, 2, 7, 1], [6, 3, 6, 2, 2, 1, 6, 7], [0, 0, 6, 0, 7, 3, 5, 3], [7, 7, 6, 6, 3, 2, 0, 3], [7, 0, 0, 5, 4, 2, 2, 3], [7, 2, 3, 0, 5, 3,

  Evaluated 56 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.47
  Std 1.8732591918898998
-- Generation 50 --
  Evaluated 63 individuals----------------
  Min 0.0
  Max 11.0
  Avg 0.33
  Std 1.5559884318336046
-- Generation 51 --
  Evaluated 57 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.34
  Std 1.4981321704042003
-- Generation 52 --
  Evaluated 74 individuals----------------
  Min 0.0
  Max 6.0
  Avg 0.22
  Std 0.9754998718605759
-- Generation 53 --
  Evaluated 54 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.46
  Std 1.6936351437071682
-- Generation 54 --
  Evaluated 67 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.76
  Std 2.3070327262524906
-- Generation 55 --
  Evaluated 59 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.49
  Std 1.8573906428104994
-- Generation 56 --
  Evaluated 65 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.2
  Std 1.1832159566199232
-- Generation 57 --
  Evaluated 56 individuals----------------
  

  Evaluated 64 individuals----------------
  Min 0.0
  Max 11.0
  Avg 0.45
  Std 1.745709025009609
-- Generation 132 --
  Evaluated 53 individuals----------------
  Min 0.0
  Max 9.0
  Avg 0.33
  Std 1.4903355326905414
-- Generation 133 --
  Evaluated 62 individuals----------------
  Min 0.0
  Max 6.0
  Avg 0.2
  Std 1.0
-- Generation 134 --
  Evaluated 61 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.25
  Std 1.5580436450882884
-- Generation 135 --
  Evaluated 65 individuals----------------
  Min 0.0
  Max 11.0
  Avg 0.55
  Std 1.7965244223221681
-- Generation 136 --
  Evaluated 69 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.22
  Std 1.127652428720836
-- Generation 137 --
  Evaluated 52 individuals----------------
  Min 0.0
  Max 11.0
  Avg 0.35
  Std 1.5124483462254175
-- Generation 138 --
  Evaluated 54 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.4
  Std 1.7832554500127007
-- Generation 139 --
  Evaluated 67 individuals----------------
  Min 0.0
 

  Evaluated 57 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.43
  Std 1.7734429790664261
-- Generation 207 --
  Evaluated 53 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.43
  Std 1.8614779074702983
-- Generation 208 --
  Evaluated 62 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.38
  Std 1.7987773625437917
-- Generation 209 --
  Evaluated 58 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.5
  Std 1.6340134638368191
-- Generation 210 --
  Evaluated 55 individuals----------------
  Min 0.0
  Max 11.0
  Avg 0.61
  Std 1.9385303711832838
-- Generation 211 --
  Evaluated 59 individuals----------------
  Min 0.0
  Max 11.0
  Avg 0.44
  Std 1.8618270596379247
-- Generation 212 --
  Evaluated 58 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.36
  Std 1.48
-- Generation 213 --
  Evaluated 57 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.65
  Std 2.113646138784825
-- Generation 214 --
  Evaluated 64 individuals----------------
  Min 0

  Evaluated 57 individuals----------------
  Min 0.0
  Max 6.0
  Avg 0.32
  Std 1.1906300852909772
-- Generation 277 --
  Evaluated 66 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.42
  Std 1.5822768405054788
-- Generation 278 --
  Evaluated 73 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.51
  Std 1.9874355335456795
-- Generation 279 --
  Evaluated 57 individuals----------------
  Min 0.0
  Max 11.0
  Avg 0.35
  Std 1.6147755261955143
-- Generation 280 --
  Evaluated 61 individuals----------------
  Min 0.0
  Max 18.0
  Avg 0.49
  Std 2.3086576186173646
-- Generation 281 --
  Evaluated 57 individuals----------------
  Min 0.0
  Max 20.0
  Avg 0.78
  Std 2.5906755875639855
-- Generation 282 --
  Evaluated 67 individuals----------------
  Min 0.0
  Max 15.0
  Avg 0.62
  Std 2.4485914318236106
-- Generation 283 --
  Evaluated 58 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.44
  Std 2.084802148886076
-- Generation 284 --
  Evaluated 65 individuals---------

...

  Evaluated 71 individuals----------------
  Min 0.0
  Max 11.0
  Avg 0.59
  Std 1.9753227584372128
-- Generation 677 --
  Evaluated 71 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.6
  Std 2.2045407685048604
-- Generation 678 --
  Evaluated 51 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.24
  Std 1.1056219968868202
-- Generation 679 --
  Evaluated 60 individuals----------------
  Min 0.0
  Max 9.0
  Avg 0.47
  Std 1.8732591918898998
-- Generation 680 --
  Evaluated 55 individuals----------------
  Min 0.0
  Max 15.0
  Avg 0.61
  Std 2.097117068739845
-- Generation 681 --
  Evaluated 61 individuals----------------
  Min 0.0
  Max 24.0
  Avg 0.81
  Std 3.1102893756047845
-- Generation 682 --
  Evaluated 67 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.89
  Std 2.6604322956993287
-- Generation 683 --
  Evaluated 54 individuals----------------
  Min 0.0
  Max 9.0
  Avg 0.27
  Std 1.3479985163196582
-- Generation 684 --
  Evaluated 57 individuals-----------

  Evaluated 56 individuals----------------
  Min 0.0
  Max 15.0
  Avg 0.27
  Std 1.7254274832632055
-- Generation 752 --
  Evaluated 58 individuals----------------
  Min 0.0
  Max 6.0
  Avg 0.2
  Std 1.0
-- Generation 753 --
  Evaluated 61 individuals----------------
  Min 0.0
  Max 15.0
  Avg 0.66
  Std 2.5108564275959706
-- Generation 754 --
  Evaluated 62 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.4
  Std 1.624807680927192
-- Generation 755 --
  Evaluated 62 individuals----------------
  Min 0.0
  Max 20.0
  Avg 0.76
  Std 2.6043041297052847
-- Generation 756 --
  Evaluated 64 individuals----------------
  Min 0.0
  Max 15.0
  Avg 0.6
  Std 2.328089345364563
-- Generation 757 --
  Evaluated 73 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.37
  Std 1.7530259553126988
-- Generation 758 --
  Evaluated 57 individuals----------------
  Min 0.0
  Max 15.0
  Avg 0.52
  Std 2.160925727552893
-- Generation 759 --
  Evaluated 70 individuals----------------
  Min 0.0
  

  Evaluated 68 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.42
  Std 1.5045265035884212
-- Generation 822 --
  Evaluated 50 individuals----------------
  Min 0.0
  Max 6.0
  Avg 0.3
  Std 1.307669683062202
-- Generation 823 --
  Evaluated 48 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.48
  Std 1.6998823488700623
-- Generation 824 --
  Evaluated 52 individuals----------------
  Min 0.0
  Max 11.0
  Avg 0.27
  Std 1.4202464574854603
-- Generation 825 --
  Evaluated 57 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.46
  Std 1.9205207627099479
-- Generation 826 --
  Evaluated 71 individuals----------------
  Min 0.0
  Max 6.0
  Avg 0.1
  Std 0.714142842854285
-- Generation 827 --
  Evaluated 49 individuals----------------
  Min 0.0
  Max 6.0
  Avg 0.16
  Std 0.9243376006633075
-- Generation 828 --
  Evaluated 64 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.48
  Std 1.676186147180557
-- Generation 829 --
  Evaluated 62 individuals----------------


  Evaluated 55 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.54
  Std 1.7630655121123548
-- Generation 896 --
  Evaluated 60 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.2
  Std 1.1489125293076057
-- Generation 897 --
  Evaluated 55 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.43
  Std 1.7507426995421116
-- Generation 898 --
  Evaluated 65 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.38
  Std 1.4613692209705251
-- Generation 899 --
  Evaluated 59 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.4
  Std 1.624807680927192
-- Generation 900 --
  Evaluated 62 individuals----------------
  Min 0.0
  Max 15.0
  Avg 0.67
  Std 2.163584987930911
-- Generation 901 --
  Evaluated 72 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.67
  Std 2.345442388974839
-- Generation 902 --
  Evaluated 52 individuals----------------
  Min 0.0
  Max 24.0
  Avg 0.89
  Std 3.2338676534453294
-- Generation 903 --
  Evaluated 53 individuals---------------

  Evaluated 63 individuals----------------
  Min 0.0
  Max 15.0
  Avg 0.39
  Std 1.902077811236964
-- Generation 970 --
  Evaluated 66 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.38
  Std 1.6660132052297785
-- Generation 971 --
  Evaluated 65 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.28
  Std 1.3862178760930766
-- Generation 972 --
  Evaluated 62 individuals----------------
  Min 0.0
  Max 22.0
  Avg 0.5
  Std 2.567099530598687
-- Generation 973 --
  Evaluated 76 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.81
  Std 2.663437628329224
-- Generation 974 --
  Evaluated 57 individuals----------------
  Min 0.0
  Max 13.0
  Avg 0.66
  Std 2.3117958387366304
-- Generation 975 --
  Evaluated 58 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.44
  Std 1.75681530047982
-- Generation 976 --
  Evaluated 62 individuals----------------
  Min 0.0
  Max 8.0
  Avg 0.44
  Std 1.6632498309033432
-- Generation 977 --
  Evaluated 64 individuals----------------